In [30]:
import tensorflow as tf
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [31]:
# Constants
IMAGE_SIZE = (128, 128)  # Resize images for consistency (you can experiment with different sizes)
BATCH_SIZE = 32
NUM_CLASSES = 26  # Assuming you're classifying 26 letters

In [32]:
# Path to the image directory (adjust to your dataset location)
IMAGE_DIR = 'flipped_archive/asl_alphabet_train/'  # Adjust this to your dataset directory path

In [33]:
# ImageDataGenerator for loading and augmenting images
datagen = ImageDataGenerator(rescale=1./255,  # Normalize pixel values to [0, 1]
                             validation_split=0.2,  # Split 20% of data for validation
                             rotation_range=30,  # Random rotations
                             width_shift_range=0.2,  # Horizontal shifts
                             height_shift_range=0.2,  # Vertical shifts
                             shear_range=0.2,  # Random shear
                             zoom_range=0.2,  # Random zoom
                             fill_mode='nearest')  # Fill missing pixels after transforms

In [34]:
# Train and validation data generators
train_generator = datagen.flow_from_directory(
    IMAGE_DIR,
    target_size=IMAGE_SIZE,  # Resize images
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Multi-class classification
    subset='training')  # Use the training subset

val_generator = datagen.flow_from_directory(
    IMAGE_DIR,
    target_size=IMAGE_SIZE,  # Resize images
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Multi-class classification
    subset='validation')  # Use the validation subset

Found 329830 images belonging to 26 classes.
Found 82444 images belonging to 26 classes.


In [35]:
# Step 2: Build the CNN model

def build_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),  # Input layer (64x64 RGB images)
        
        # First convolutional block
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Second convolutional block
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Third convolutional block
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Flatten the output and add fully connected layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Dropout to avoid overfitting
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')  # Output layer (26 classes)
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [36]:
# Build the model
model = build_cnn_model()

In [37]:
# Step 3: Set up callbacks (EarlyStopping and ReduceLROnPlateau)
es_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

In [38]:
# Step 4: Train the model

history = model.fit(
    train_generator,
    epochs=50,  # Adjust epochs based on your dataset
    validation_data=val_generator,
    callbacks=[es_callback, lr_callback]
)

Epoch 1/50
10308/10308 ━━━━━━━━━━━━━━━━━━━━ 3352s 325ms/step - accuracy: 0.3062 - loss: 2.2621 - val_accuracy: 0.6680 - val_loss: 1.0799 - learning_rate: 0.0010
Epoch 2/50
10308/10308 ━━━━━━━━━━━━━━━━━━━━ 3304s 321ms/step - accuracy: 0.6946 - loss: 0.9281 - val_accuracy: 0.7436 - val_loss: 0.8405 - learning_rate: 0.0010
Epoch 3/50
10308/10308 ━━━━━━━━━━━━━━━━━━━━ 3319s 322ms/step - accuracy: 0.7599 - loss: 0.7310 - val_accuracy: 0.7767 - val_loss: 0.7243 - learning_rate: 0.0010
Epoch 4/50
10308/10308 ━━━━━━━━━━━━━━━━━━━━ 3372s 327ms/step - accuracy: 0.7925 - loss: 0.6409 - val_accuracy: 0.7692 - val_loss: 0.7595 - learning_rate: 0.0010
Epoch 5/50
10308/10308 ━━━━━━━━━━━━━━━━━━━━ 3397s 330ms/step - accuracy: 0.8067 - loss: 0.5973 - val_accuracy: 0.7719 - val_loss: 0.7454 - learning_rate: 0.0010
Epoch 6/50
10308/10308 ━━━━━━━━━━━━━━━━━━━━ 3370s 327ms/step - accuracy: 0.8177 - loss: 0.5669 - val_accuracy: 0.7972 - val_loss: 0.6788 - learning_rate: 0.0010
Epoch 7/50
10308/10308 ━━━━━━━━━━━

KeyboardInterrupt: 

In [39]:
# Save the model
timestamp = time.strftime("%Y%m%d-%H%M%S")
model.save(f'path_to_save_model/alphabet_hand_model_{timestamp}.h5')

In [40]:
# Step 5: Evaluate the model (after training)
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

2577/2577 ━━━━━━━━━━━━━━━━━━━━ 349s 135ms/step - accuracy: 0.8197 - loss: 0.6190
Validation Loss: 0.6173076629638672
Validation Accuracy: 0.8205084800720215
